Few days ago, I have shared notebook which denoise using sound envelope.(https://www.kaggle.com/takamichitoda/birdcall-noise-reduction)

But if submit using this method, timeout error is occured.

In this notebook, I share that reduce to processing time to submit in time.

This notebook is made from [Hidehisa's notebook](https://www.kaggle.com/hidehisaarai1213/inference-pytorch-birdcall-resnet-baseline), and and noisereduce library is loaded from [here](https://www.kaggle.com/ajax0564/noisereduce).
Thank you Hidehisa Arai and ankit maurya!

In [ ]:
import cv2
import audioread
import logging
import os
import random
import time
import warnings

import IPython
import librosa
import numpy as np
import pandas as pd
import soundfile as sf
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data as data

from torchvision.models import resnet18, resnet50, densenet121, densenet161

from contextlib import contextmanager
from pathlib import Path
from typing import Optional

from fastprogress import progress_bar
from sklearn.metrics import f1_score
from torchvision import models
from matplotlib import pyplot as plt

from torchvision.transforms.functional import to_tensor
from torchvision.transforms import Normalize

import time
from datetime import timedelta as td
from scipy.ndimage import maximum_filter1d
import scipy

device = torch.device("cuda")
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

Set my configuration and load dataset.

In [ ]:
class config:
    TARGET_SR = 32000
    MELSPECTROGRAM_PARAMETERS = {"n_mels": 128, "fmin": 20, "fmax": 16000}
    SEED = 416
    N_LABEL = 264
    PRETRAINED = False
    THRESHOLD = 0.5
    WEIGHTS_PATH = "../input/birdcall-densenet161/birdcallnet_f0_densenet161.bin"
    SED_THRESHOLD = 0.05
    ENVELOPE = 0.02
    

# Get Test Set
TEST = Path("../input/birdsong-recognition/test_audio").exists()
if TEST:
    DATA_DIR = Path("../input/birdsong-recognition/")
else:
    # dataset created by @shonenkov, thanks!
    DATA_DIR = Path("../input/birdcall-check/")
test = pd.read_csv(DATA_DIR / "test.csv")
test_audio = DATA_DIR / "test_audio"
sub = pd.read_csv("../input/birdsong-recognition/sample_submission.csv")
sub.to_csv("submission.csv", index=False)


# Get BIRD_CODE dict
train_df = pd.read_csv('../input/birdsong-recognition/train.csv')
keys = set(train_df.ebird_code)
values = np.arange(0, len(keys))
code_dict = dict(zip(sorted(keys), values))
n_labels = len(code_dict)
INV_BIRD_CODE = {v: k for k, v in code_dict.items()}

My model is Densenet161. It local fold-0 f1 score is 0.685494403 and LB score is 0.471.

In [ ]:
class BirdcallNet(nn.Module):
    def __init__(self):
        super(BirdcallNet, self).__init__()
        self.densenet = densenet161(pretrained=config.PRETRAINED)
        self.densenet.classifier = nn.Linear(2208, config.N_LABEL)

    def forward(self, x):
        return self.densenet(x)

install and load noisereduce.

In [ ]:
!pip install /kaggle/input/noisereduce/noisereduce-1.0.1-py3-none-any.whl

In [ ]:
import noisereduce as nr


Let's try noise reduce and check birdcall.

In [ ]:
ebird_code, filename = train_df.sample(1, random_state=123)[["ebird_code", "filename"]].values[0]
path = f"../input/birdsong-recognition/train_audio/{ebird_code}/{filename}"

x, sr = librosa.load(path, mono=True, res_type="kaiser_fast")

print("Sampling Rate:", sr)
plt.plot(x);

I detect point no birdcall by using Sound Envelope.

I reffered [this notebook](https://www.kaggle.com/jainarindam/imp-remove-background-dead-noise).
Thank you Arindam!

In [ ]:
def envelope(y, rate, threshold):
    y_mean = maximum_filter1d(np.abs(y), mode="constant", size=rate//20)
    mask = [mean > threshold for mean in y_mean]
    return mask

I use initial 2 second to reduce processing time.

In [ ]:
initial_2s = x[:sr*2]
plt.plot(initial_2s)

In [ ]:
mask = envelope(initial_2s, sr, config.ENVELOPE)  # config.ENVELOPE = 0.02
plt.plot(mask)

In [ ]:
x_denoise = nr.reduce_noise(audio_clip=x, noise_clip=initial_2s[np.logical_not(mask)], verbose=True)

Let's compare treated audio

In [ ]:
# original
IPython.display.Audio(data=x, rate=sr)

In [ ]:
# denoise
IPython.display.Audio(data=x_denoise, rate=sr)

Even if initial few second, It seems effective enough.

### Predict test data with denoise.

Dataset Class

In [ ]:
def mono_to_color(X, mean=None, std=None, norm_max=None, norm_min=None, eps=1e-6):
    # Stack X as [X,X,X]
    X = np.stack([X, X, X], axis=-1)

    # Standardize
    mean = mean or X.mean()
    X = X - mean
    std = std or X.std()
    Xstd = X / (std + eps)
    _min, _max = Xstd.min(), Xstd.max()
    norm_max = norm_max or _max
    norm_min = norm_min or _min
    if (_max - _min) > eps:
        # Normalize to [0, 255]
        V = Xstd
        V[V < norm_min] = norm_min
        V[V > norm_max] = norm_max
        V = 255 * (V - norm_min) / (norm_max - norm_min)
        V = V.astype(np.uint8)
    else:
        # Just zero
        V = np.zeros_like(Xstd, dtype=np.uint8)
    return V


class TestDataset(data.Dataset):
    def __init__(self, df, clip):
        self.df = df
        self.clip = clip
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx: int):
        SR = config.TARGET_SR
        sample = self.df.loc[idx, :]
        site = sample.site
        row_id = sample.row_id
        
        if site == "site_3":
            y = self.clip.astype(np.float32)
            len_y = len(y)
            start = 0
            end = SR * 5
            images = []
            while len_y > start:
                y_batch = y[start:end].astype(np.float32)
                if len(y_batch) != (SR * 5):
                    break

                # Denoise
                _y_batch = y_batch[:config.TARGET_SR*2]
                mask = envelope(_y_batch, config.TARGET_SR, threshold=config.ENVELOPE)
                noise_clip = _y_batch[np.logical_not(mask)]
                if len(noise_clip):  # noise is exist
                    x_denoise = nr.reduce_noise(y_batch, noise_clip)
                    y_batch = x_denoise
                      
                start = end
                end = end + SR * 5
 
                melspec = librosa.feature.melspectrogram(y_batch,
                                                         sr=SR,
                                                         **config.MELSPECTROGRAM_PARAMETERS)
                melspec = librosa.power_to_db(melspec).astype(np.float32)
                image = mono_to_color(melspec)
                image = to_tensor(image)
                image = Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))(image)
                image = image.numpy()
                images.append(image)
            images = np.asarray(images)
            return images, row_id, site
        else:
            end_seconds = int(sample.seconds)
            start_seconds = int(end_seconds - 5)
            
            start_index = SR * start_seconds
            end_index = SR * end_seconds
            
            y = self.clip[start_index:end_index].astype(np.float32)
            
            # denoise
            _y = y[:config.TARGET_SR*2]
            mask = envelope(_y, config.TARGET_SR, threshold=config.ENVELOPE)
            noise_clip = _y[np.logical_not(mask)]
            if len(noise_clip):  # noise is exist
                x_denoise = nr.reduce_noise(y, noise_clip)
                y = x_denoise
                
            melspec = librosa.feature.melspectrogram(y, sr=SR, **config.MELSPECTROGRAM_PARAMETERS)
            melspec = librosa.power_to_db(melspec).astype(np.float32)

            image = mono_to_color(melspec)
            image = to_tensor(image)
            image = Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))(image)
            image = image.numpy()

            return image, row_id, site

Predict Function

In [ ]:
def prediction_for_clip(test_df, clip, model):

    dataset = TestDataset(df=test_df, clip=clip)
    loader = data.DataLoader(dataset, batch_size=1, shuffle=False)
    
    model.eval()
    prediction_dict = {}
    for image, row_id, site in progress_bar(loader):
        site = site[0]
        row_id = row_id[0]
        if site in {"site_1", "site_2"}:
            image = image.to(device)
            
            if image.sum() == 0:
                labels = []    
            else:
                with torch.no_grad():
                    prediction = model(image)
                proba = prediction.detach().cpu().sigmoid().numpy().reshape(-1)
                events = proba >= config.THRESHOLD
                labels = np.argwhere(events).reshape(-1).tolist()
                
        else:
            image = image.squeeze(0)
            batch_size = 16
            whole_size = image.size(0)
            if whole_size % batch_size == 0:
                n_iter = whole_size // batch_size
            else:
                n_iter = whole_size // batch_size + 1
                
            all_events = set()
            for batch_i in range(n_iter):
                batch = image[batch_i * batch_size:(batch_i + 1) * batch_size]
                if batch.ndim == 3:
                    batch = batch.unsqueeze(0)
                
                batch = batch.to(device)
                with torch.no_grad():
                    prediction = model(batch)
                    proba = prediction.detach().cpu().sigmoid().numpy()
                    
                events = proba >= config.THRESHOLD
                for i in range(len(events)):
                    event = events[i, :]
                    labels = np.argwhere(event).reshape(-1).tolist()
                    for label in labels:
                        all_events.add(label)
                        
            labels = list(all_events)
        if len(labels) == 0:
            prediction_dict[row_id] = "nocall"
        else:
            labels_str_list = list(map(lambda x: INV_BIRD_CODE[x], labels))
            label_string = " ".join(labels_str_list)
            prediction_dict[row_id] = label_string
    return prediction_dict

def prediction(test_df, test_audio):
    
    model = BirdcallNet()
    model.load_state_dict(torch.load(config.WEIGHTS_PATH))
    model.to(device)
    model.eval()
    
    unique_audio_id = test_df.audio_id.unique()


    prediction_dfs = []
    for audio_id in unique_audio_id:
        clip, _ = librosa.load(test_audio / (audio_id + ".mp3"),
                               sr=config.TARGET_SR,
                               mono=True,
                               res_type="kaiser_fast")
         
        test_df_for_audio_id = test_df.query(f"audio_id == '{audio_id}'").reset_index(drop=True)
        prediction_dict = prediction_for_clip(test_df_for_audio_id, clip=clip, model=model)
        
        row_id = list(prediction_dict.keys())
        birds = list(prediction_dict.values())
            
        prediction_df = pd.DataFrame({
            "row_id": row_id,
            "birds": birds
        })
        prediction_dfs.append(prediction_df)
    
    prediction_df = pd.concat(prediction_dfs, axis=0, sort=False).reset_index(drop=True)
    return prediction_df

In [ ]:
submission = prediction(test_df=test, test_audio=test_audio)
submission.to_csv("submission.csv", index=False)

In [ ]:
display(submission)

This result can submit in time. 

But LB score may be not good.

The one of reason is that the model was trained without denoise processing. We should apply same processing in train time.